In [2]:

using Pkg
Pkg.add("Plots")
using Plots
using Optim


    Updating registry at `C:\Users\bensp\.julia\registries\General`


In [ ]:
struct Params
        m::Float64       # kg
        ρ_air::Float64       # kg/m^3
        ρ_water::Float64       # kg/m^3
        g::Float64       # m/s^2
        GMₜ::Float64     # m
        Ixx::Float64     # kg m^2
        Kₚ::Float64      # kg m^2
        ∇::Float64       # m^3, calculated as m/ρ
        h2::Float64      # m
        A::Float64       # m^2, sail area
        l1::Float64      # m, distance from the center of mass to the sail
        l2::Float64      # m, distance from the center of mass to the rudder
        h::Float64       # m, height of the sail
    end
    
    params = Params(
        1600,       # m
        1020,       # ρ_air
        1.225,      # ρ_water
        9.81,       # g
        2.4,        # GMₜ
        6800,       # Ixx
        -1000,      # Kₚ
        1600 / 1020, # ∇, calculated as m/ρ
        0.95,       # h2
        1,          # A
        1,          # l1
        1,          # l2
        1           # h
    )
    


In [ ]:
# Refering the following paper for the model
# Roll Stabilization Control of Sailboats
# https://www.sciencedirect.com/science/article/pii/S2405896316320833

# Assuming Constant Wind Speed
V_ws = 5.0  # m/s, example constant wind speed
# Setting some global variables for the sailboat


function angle_of_attack(λ, βws)
    # Calculate the angle of attack
    α = βws - λ + π
    return α
end

# Taken from wikipedia page on lift and drag coefficients
# https://en.wikipedia.org/wiki/Forces_on_sails
function CL(α)
    α_d = rad2deg(α)  # Convert angle from radians to degrees for the calculations
    if α_d <= 10
        return 0.0
    elseif α_d <= 25
        # Linearly increase CL from 0 at 10 degrees to 1.5 at 25 degrees
        return (α_d - 10) * (1.5 / (25 - 10)) * 10
    else
        # Linearly decrease CL from 1.5 at 25 degrees to 0.2 at 100 degrees
        return max(1.5 - (α_d - 25) * ((1.5 - 0.2) / (100 - 25)), 0.2) * 10
    end
end

# Drag coefficient CD as a function of angle of attack α in radians
function CD(α)
    α_d = rad2deg(α)  
    # Quadratic increase to reach 1.6 at 100 degrees
    return min((α_d^2) * (1.6 / 100^2), 1.7) * 10
end

α_range = range(0, stop=π/1.5, length=200)
CL_values = [CL(α) for α in α_range]
CD_values = [CD(α) for α in α_range]

# Plot the coefficients to visualize the lift and drag
p = plot(rad2deg.(α_range), CL_values, label="C_L", xlabel="Angle of Attack (degrees)", ylabel="Coefficient", title="Lift and Drag Coefficients", ylims=(0,20))
plot!(rad2deg.(α_range), CD_values, label="C_D", ylims=(0,20))
display(p)


In [ ]:

function SL_(params, Vws,λ, βws)
    # Calculate the lift force
    A = params.A
    ρ = params.ρ_air
    Fl = 0.5 * ρ * Vws^2 * A * CL(angle_of_attack(λ, βws))
    return Fl

end


function SD_(params,Vws, λ, βws)
    A = params.A
    # Calculate the drag force
    ρ = params.ρ_air
    Fd = 0.5 * ρ * Vws^2 * A * CD(angle_of_attack(λ, βws))
    return Fd
end


function Sail_forces(params, Vws,λ, βws)

    # Parameters
    l1 = params.l1
    l2 = params.l2
    h = params.h

    # Calculate the forces on the sail
    SL = SL_(params, Vws,λ, βws)
    SD = SD_(params, Vws,λ, βws)
    S1 = -SL * sin(βws) + SD * cos(βws)
    S2 = SL * cos(βws) + SD * sin(βws)
    S3 = h*S2
    S4 = S1*(l2 * sin(λ))- S2*(l1 + l2 * cos(λ))

    S = [S1, S2, S3, S4]'
    return S
end

# Function to calculate S'_{x,r}
function S_prime_x_r(params, Vws,λ, βws)
    S = Sail_forces(params, Vws,λ, βws)
    # Here S[1] represents S1, the force in the surge direction
    S_prime_x_r = S[1] / Vws^2
    return S_prime_x_r
end



# To find the optimal sail angle, we will optimize over S'_x_r for a given wind speed to find the optimal sail angle defined: λ_opt


In [ ]:
Vws = 5.0  # m/s, example constant wind speed
# Create ranges for relative wind angle and sail angle
βws_values = range(deg2rad(-180), deg2rad(180), length=100)  # Relative wind angle
λ_values = range(deg2rad(-180), deg2rad(180), length=100)    # Sail angle

# Compute the normalized forces for the contour plot
Z = [S_prime_x_r(params, Vws,λ, βws) for βws in βws_values, λ in λ_values]

# Create a contour plot of the relative forward force
p = contour(βws_values .* 180 / π, λ_values .* 180 / π, Z,
            title="Relative Forward Force",
            xlabel="Relative Wind Angle (degrees)",
            ylabel="Sail Angle (degrees)",
            fill=true)

# Display the plot
display(p)





In [ ]:

# Defining the bounds for the sail angle λ based on the wind angle βws and the saturation limit λ_sat
function λ_bounds(βws, λ_sat)
    if βws < 0
        λ_u = βws + π
        λ_l = 0
    elseif βws > 0
        λ_u = 0
        λ_l = βws - π
    else # Handling the case when βws is 0
        λ_u = 0
        λ_l = -π
    end
    # Apply saturation limits
    λ_u = min(λ_u, λ_sat)
    λ_l = max(λ_l, -λ_sat)
    return λ_l, λ_u
end

function compute_lookup_table(params, V_ws, βws_values, λ_sat, verbose=false)
    # Extract the parameters
    A = params.A
    h = params.h
    l1 = params.l1
    l2 = params.l2
    

    # Create a lookup table to store the optimal sail angle for each wind angle
    λ_opt_values = Dict()

    # Iterate through all wind angles βws
    for βws in βws_values
        # Get the bounds for λ
        λ_l, λ_u = λ_bounds(βws, λ_sat)
        

        # Optimize to find λ that maximizes S'_x_r within the bounds
        opt_result = optimize(λ -> -S_prime_x_r(params, Vws,λ, βws), λ_l, λ_u)
        # opt_result = optimize(λ -> -S_prime_x_r(V_ws, A, λ, βws, h, l1, l2), -π, π)
        λ_opt = opt_result.minimizer
        if verbose
            force = S_prime_x_r(params, Vws,λ, βws)
            println("βws: ", βws, " λ_l: ", λ_l, " λ_u: ", λ_u, " λ_opt: ", λ_opt, " S'_x_r: ", force)
        end
        # Store the result in the lookup table
        λ_opt_values[βws] = λ_opt
    end
    return λ_opt_values
end


In [ ]:
λ_sat = π / 2  # standatd saturation limit for the sail angle
βws_values = range(deg2rad(-180), stop=deg2rad(0), length=360) # Wind angles from -180 to 180 degrees in radians

optimal_angle = compute_lookup_table(params, V_ws, βws_values, λ_sat, false)


In [ ]:

βws_degrees = rad2deg.(collect(keys(optimal_angle)))  # Convert wind direction from radians to degrees
λ_opt_degrees = rad2deg.(collect(values(optimal_angle)))  # Convert sail angle from radians to degrees


sorted_indices = sortperm(βws_degrees)
βws_degrees_sorted = βws_degrees[sorted_indices]
λ_opt_degrees_sorted = λ_opt_degrees[sorted_indices]


line_plot = plot(βws_degrees_sorted, λ_opt_degrees_sorted, title="Optimal Sail Angle vs Wind Direction", xlabel="Relative Wind Direction (degrees)", ylabel="Optimal Sail Angle (degrees)", legend=false, linewidth=2)
display(line_plot)

# This plot follows similarly to the one in the paper, although their lift and drag coefficients seem more refined. They reference a paywall text for the coefficients, so I am unable to access it.
# Assume symetry in the sailboat
